# Structural Estimation: Monte Carlo

In [1]:
using Pkg
Pkg.activate(".")
#for pkg in ["BlackBoxOptim", "Cubature", "Distributions", "Integrals", "Roots", "PrettyTables", "JLD", "StatProfilerHTML"] # 
#    Pkg.add(pkg)
#end

Pkg.instantiate()

using Base.Threads, BlackBoxOptim, Cubature, Dates, Distributions, Integrals, Random, Roots, PrettyTables, JLD

  Activating project at `C:\Users\jbrig\Documents\research\mapinator_2024\notes\estimation_of_tau`


In [2]:
estimated_parameters = load("estimated_parameters_collapse_sinks_new.jld")

Dict{String, Any} with 9 entries:
  "sol_2"           => [2.48187, 0.476956, 0.429753, -3.75801, 0.358705, 0.2595…
  "estimated_α"     => [0.1029, 0.140406, 0.173084, 0.0882224, 0.0520296]
  "estimated_n_val" => 25890.1
  "estimated_τ"     => 0.556643
  "k"               => 4
  "estimated_v_rel" => [0.512286, 0.544467, 0.734376]
  "estimated_γ"     => [0.184859, 0.252238, 0.310943, 0.15849, 0.0934703]
  "K"               => 7
  "m_t_values"      => Any[2664, 3635, 4481, 2284, 1347]

In [3]:
estimated_sol_2 = estimated_parameters["sol_2"]
estimated_v_rel = estimated_parameters["estimated_v_rel"]
estimated_α = estimated_parameters["estimated_α"]
estimated_τ = estimated_parameters["estimated_τ"]
k = estimated_parameters["k"]
K = estimated_parameters["K"]
m_t_values = estimated_parameters["m_t_values"]
estimated_m_val = sum(m_t_values)
estimated_n_val = (estimated_m_val / estimated_τ) + 1
estimated_m_val

14411

In [4]:
estimated_γ = estimated_parameters["estimated_γ"]

5-element Vector{Float64}:
 0.1848587884255083
 0.25223787384636737
 0.31094302963014364
 0.15849004232877664
 0.09347026576920409

In [5]:
# define p_vec as e.g.
# [v2/v1 v3/v2 v4/v3 α1 α2 α3 α4 mu1 mu2 mu3 mu4 mus sg1 sg2 sg3 sg4 sgs]

function q_prob(t, v_rel, mkt_balance_ratio, gamma, k)
    numerator = prod([v_rel[s] for s in t:k-1])
    denominator = exp(1 / mkt_balance_ratio) * prod([((v_rel[s])^(sum(gamma[1:s]))) for s in 1:k-1])
    return numerator / denominator
end

function F(x, ρ, normals, K)
    sum_base = 0.0
    for i in 1:K
        sum_base += ρ[i] * cdf(normals[i], x)
    end
    return sum_base
end

function f_integrand(integrals, x, p) 
    base_exp = exp(F(x, p.ρ, p.normals, p.K) / sum(p.α[1:p.s]))
    for i in 1:p.K
        integrals[i] = base_exp * pdf(p.normals[i], x)
    end
end

function get_integrals(x_vec, ρ, normals, α, k, K)
    # Fx_vec = [F(x0)=1, F(x1), F(x2), F(x3), ..., F(xk-1)]
    # x_vec = [x0 = 1, x1, x2, x3, ..., xk = 0]
    # x_vec[s] = x_{s-1}, so the limits of integration are and must be offset by 1 below
    all_integrals = zeros(K, k)
    for s in 1:k
        # https://docs.sciml.ai/Integrals/stable/basics/SampledIntegralProblem/ might be faster
        # if f_integrand(x) can be vectorized/sped up
        prob = IntegralProblem(IntegralFunction(f_integrand, zeros(K)), (x_vec[s+1], x_vec[s]), (; s, ρ, normals, α, K))
        sol = solve(prob, CubatureJLh(); reltol = 1e-3, abstol = 1e-3)
        integrals_result = sol.u
        # NOTE: result may be inf if alpha_1 is too small
        # NOTE: some parameter values for μ and σ may cause the cdf F_i to be NaN
        for i in 1:K 
            all_integrals[i, s] = integrals_result[i]
        end
    end
    return all_integrals
end      

function q(i, t, all_integrals, Fx_vec, α, v_rel, k)
    return α[t] * sum([(1/sum(α[1:s])) * prod([v_rel[j] for j in t:(s-1)]) * exp(-Fx_vec[s] / sum(α[1:s])) * all_integrals[i, s] for s in t:k])
end

function Fx(t, α, v_rel)
    return 1 - sum([-log(v_rel[j])*sum(α[1:j]) for j in 1:t])
end

function pi(t, α)
    return α[t] / sum(α[1:t])
end

function ρ_q(t, α, v_rel, k)
    Fx_km1 = Fx(k-1, α, v_rel)
    if Fx_km1 <= 0
        return -1
    end
    last_term = prod([v_rel[j] for j in t:(k-1)]) * (1 - exp((0 - Fx_km1) / sum(α[1:k])))
    return α[t] * (sum([prod([v_rel[j] for j in t:(s-1)]) * (1 - v_rel[s]) for s in t:(k-1)]) + last_term)
end

ρ_q (generic function with 1 method)

In [6]:
function estimate_parameters_stage_1(p_vec, placements, γ, m_val, k)
    v_rel = p_vec[1:k-1]
    τ = p_vec[k]
    α = τ * γ

    n_val = (m_val / τ) + 1

    objective = 0.0

    ρ_q_t = zeros(k)
    for t in 1:k
        prob = ρ_q(t, α, v_rel, k)
        if prob < 0
            return Inf
        end
        ρ_q_t[t] = prob
    end

    for t in 1:k
        prob = ρ_q_t[t]
        expectation = n_val * prob
        objective += ((sum(placements[:, t]) - expectation) ^ 2) / expectation
    end
    
    return objective
end

estimate_parameters_stage_1 (generic function with 1 method)

In [7]:
function estimate_likelihood_2(p_vec, v_rel, α, placements, k, K, counter)   
    μ = p_vec[1:K]
    σ = p_vec[K+1:2K]
    ρ_vec = p_vec[2K+1:3K]
    ρ = ρ_vec / sum(ρ_vec)

    ## compute the cutoffs x and the CDF values F(x)
    normals = [truncated(Normal(μ[i], σ[i]), 0, 1) for i in 1:K]

    Fx_vec = ones(k) # sets F(x0) = 1 by default; Fx_vec = [F(x0)=1, F(x1), F(x2), F(x3), ..., F(xk-1)]
    x_vec = ones(k+1) # x_vec = [x0 = 1, x1, x2, x3, ..., xk = 0]
    x_vec[k+1] = 0.0
    for t in 1:k-1
        Fx_vec_candidate = Fx(t, α, v_rel)
        if Fx_vec_candidate <= 0.0 # TODO: if this case occurs, can we speed up q()?
            Fx_vec[t+1:k] .= 0.0
            x_vec[t+1:k] .= 0.0
            break
        end
        Fx_vec[t+1] = Fx_vec_candidate
        # there is no simple closed-form for F^{-1}(x) so this numerically computes x1, x2, x3
        x_vec[t+1] = find_zero(x -> F(x, ρ, normals, K) - Fx_vec[t+1], 0.5) 
    end 

    ρ_q_it = zeros(K, k)
    all_integrals = get_integrals(x_vec, ρ, normals, α, k, K)
    for i in 1:K, t in 1:k
        prob = q(i, t, all_integrals, Fx_vec, α, v_rel, k)
        ρ_q_it[i, t] = ρ[i] * prob
    end

    normalizer = sum(ρ_q_it)
    likelihood = 0.0
    for i in 1:K, t in 1:k
        likelihood += placements[i, t] * log(ρ_q_it[i, t] / normalizer)
    end

    counter[1] += 1
    #if isnan(-likelihood) || isinf(-likelihood)
    #    println(all_integrals)
    #end
    return -likelihood
end

estimate_likelihood_2 (generic function with 1 method)

In [8]:
function simulate_market(p_vec, v_rel, α, k, K, n_val, m_t)
    μ = p_vec[1:K]
    σ = p_vec[K+1:2K]
    ρ_vec = p_vec[2K+1:3K]
    ρ = ρ_vec / sum(ρ_vec)

    n_i = Int.(round.(n_val * ρ))

    ## compute the cutoffs x and the CDF values F(x)
    normals = [truncated(Normal(μ[i], σ[i]), 0, 1) for i in 1:K]

    Fx_vec = ones(k) # sets F(x0) = 1 by default; Fx_vec = [F(x0)=1, F(x1), F(x2), F(x3), ..., F(xk-1)]
    x_vec = ones(k+1) # x_vec = [x0 = 1, x1, x2, x3, ..., xk = 0]
    x_vec[k+1] = 0.0
    for t in 1:k-1
        Fx_vec_candidate = Fx(t, α, v_rel)
        if Fx_vec_candidate <= 0.0 # TODO: if this case occurs, can we speed up q()?
            Fx_vec[t+1:k] .= 0.0
            x_vec[t+1:k] .= 0.0
            break
        end
        Fx_vec[t+1] = Fx_vec_candidate
        # there is no simple closed-form for F^{-1}(x) so this numerically computes x1, x2, x3
        x_vec[t+1] = find_zero(x -> F(x, ρ, normals, K) - Fx_vec[t+1], 0.5) 
    end 

    graduate_offers = []
    graduate_hiring_tiers = []
    for t in 1:k
        push!(graduate_offers, -1 * ones(m_t[t]))
        push!(graduate_hiring_tiers, -1 * ones(Int, m_t[t]))
    end
    
    for i in 1:K
        # draw wages for every department in every tier/sink
        wages = rand(normals[i], n_i[i])
        for wage in wages
            # iterate over the cutoffs
            for t in 1:k
                if wage < x_vec[t] && wage >= x_vec[t+1]
                    # determine which tier to make the offer to
                    π_j = Categorical([α[j] / sum(α[1:t]) for j in 1:t])
                    offer_tier = rand(π_j)
                    # determine which graduate within the tier to make the offer to
                    selected_graduate = rand(1:m_t[offer_tier])
                    # check for collision
                    if graduate_offers[offer_tier][selected_graduate] == -1 # no offer made to this graduate yet
                        graduate_offers[offer_tier][selected_graduate] = wage
                        graduate_hiring_tiers[offer_tier][selected_graduate] = i
                    else
                        if graduate_offers[offer_tier][selected_graduate] < wage # this offer better than existing offer
                            graduate_offers[offer_tier][selected_graduate] = wage
                            graduate_hiring_tiers[offer_tier][selected_graduate] = i
                        end
                    end 
                    break
                end
            end
        end
    end
    
    simulated_placements_matrix = zeros(Int, K, k)
    unmatched_graduates = zeros(Int, k)
    for t in 1:k
        for graduate in 1:m_t[t]
            if graduate_offers[t][graduate] == -1 # unmatched
                unmatched_graduates[t] += 1
            else # matched
                simulated_placements_matrix[graduate_hiring_tiers[t][graduate], t] += 1
            end
        end
    end

    return simulated_placements_matrix, unmatched_graduates
end

simulate_market (generic function with 1 method)

In [9]:
NUM_BOOTSTRAP_ROUNDS = 1000
NUM_ESTIMATIONS_PER_ROUND = 5

# upper/lower bound on v_rel
upper_1 = [1.0 for _ in 1:k-1]
lower_1 = [0.0 for _ in 1:k-1]

# upper/lower bound on τ
append!(upper_1, [10.0 for _ in 1:1])
append!(lower_1, [0.25 for _ in 1:1])

search_range_1 = [(lower_1[i], upper_1[i]) for i in eachindex(upper_1)]

# upper/lower bound on the mu parameter of truncated normal
upper_2 = [4.0 for _ in 1:K]
lower_2 = [-4.0 for _ in 1:K]

# upper/lower bound on the sigma parameter of truncated normal
append!(upper_2, [10.0 for _ in 1:K])
append!(lower_2, [0.12 for _ in 1:K])

# upper/lower bound on values proportional to ρ_i
append!(upper_2, [1.0 for _ in 1:K])
append!(lower_2, [0.0 for _ in 1:K])

search_range_2 = [(lower_2[i], upper_2[i]) for i in eachindex(upper_2)]

Random.seed!(0)

bootstrap_placements_output = zeros(Int, NUM_BOOTSTRAP_ROUNDS, K, k)
bootstrap_unmatched_output = zeros(Int, NUM_BOOTSTRAP_ROUNDS, k)
bootstrap_stage_1_estimates_output = zeros(NUM_BOOTSTRAP_ROUNDS, length(upper_1))
bootstrap_stage_2_estimates_output = zeros(NUM_BOOTSTRAP_ROUNDS, length(upper_2))
bootstrap_stage_1_fitness_output = zeros(NUM_BOOTSTRAP_ROUNDS)
bootstrap_stage_2_fitness_output = zeros(NUM_BOOTSTRAP_ROUNDS)
bootstrap_counter_output = zeros(Int, NUM_BOOTSTRAP_ROUNDS)

max_evals_stage_1 = 1000000
max_evals_stage_2 = 100000

Threads.@threads for bootstrap_round in 1:NUM_BOOTSTRAP_ROUNDS
    println("Starting round $bootstrap_round")
    start_time = Dates.datetime2unix(Dates.now())

    bootstrap_placements, bootstrap_unmatched = simulate_market(estimated_sol_2, estimated_v_rel, estimated_α, k, K, estimated_n_val, m_t_values)
    
    bootstrap_placements_output[bootstrap_round, :, :] = bootstrap_placements[:, :]
    bootstrap_unmatched_output[bootstrap_round, :] = bootstrap_unmatched[:]

    sol_res_1 = bboptimize(p -> estimate_parameters_stage_1(p, bootstrap_placements, estimated_γ, estimated_m_val, k), SearchRange = search_range_1, MaxFuncEvals = max_evals_stage_1, TraceMode = :silent) 
    # MaxTime = 60.0, MaxFuncEvals = 500000,
    sol_1 = best_candidate(sol_res_1)
    sol_1_fitness = best_fitness(sol_res_1)

    bootstrap_v_rel = sol_1[1:k-1]
    bootstrap_τ = sol_1[k]
    bootstrap_α = bootstrap_τ * estimated_γ
    
    bootstrap_stage_1_estimates_output[bootstrap_round, :] = sol_1[:]
    bootstrap_stage_1_fitness_output[bootstrap_round] = sol_1_fitness

    println("  round $bootstrap_round s1 best estimate: fitness $sol_1_fitness")

    best_fitness_val = Inf
    best_counter = 0
    best_sol = nothing
    stage_2_start_time = Dates.datetime2unix(Dates.now())
    for bootstrap_round_estimation in 1:NUM_ESTIMATIONS_PER_ROUND
        counter = [0]
        sol_res_2 = bboptimize(p -> estimate_likelihood_2(p, bootstrap_v_rel, bootstrap_α, bootstrap_placements, k, K, counter), SearchRange = search_range_2, MaxFuncEvals = max_evals_stage_2, TraceMode = :silent) 
        # MaxTime = 60.0, MaxFuncEvals = 500000,
        sol_2 = best_candidate(sol_res_2)
        sol_2_fitness = best_fitness(sol_res_2)
        if sol_2_fitness < best_fitness_val
            time_so_far = Dates.datetime2unix(Dates.now())
            println("    improvement in round $bootstrap_round est. $bootstrap_round_estimation: $sol_2_fitness (counter = $(counter[1])) < $best_fitness_val (counter = $best_counter); ($(time_so_far - start_time)s total)")
            best_fitness_val = sol_2_fitness
            best_counter = counter[1]
            best_sol = sol_2
        end
    end
    stage_2_end_time = Dates.datetime2unix(Dates.now())
    bootstrap_stage_2_estimates_output[bootstrap_round, :] = best_sol[:]
    bootstrap_stage_2_fitness_output[bootstrap_round] = best_fitness_val
    bootstrap_counter_output[bootstrap_round] = best_counter
    end_time = Dates.datetime2unix(Dates.now())
    println("round $bootstrap_round s2 best estimate: fitness $best_fitness_val after $best_counter iterations (runtime: $(end_time - start_time)s, avg s2 runtime: $((stage_2_end_time - stage_2_start_time)/NUM_ESTIMATIONS_PER_ROUND)s)")
end

save("monte_carlo_structure_results_collapse_sinks_1000_new.jld", "placements", bootstrap_placements_output, "unmatched", bootstrap_unmatched_output, "stage_1_estimates", bootstrap_stage_1_estimates_output, "stage_2_estimates", bootstrap_stage_2_estimates_output, "stage_1_fitness", bootstrap_stage_1_fitness_output, "stage_2_fitness", bootstrap_stage_2_fitness_output, "counter", bootstrap_counter_output, "k", k, "K", K, "max_evals_stage_1", max_evals_stage_1, "max_evals_stage_2", max_evals_stage_2, "bootstrap_rounds", NUM_BOOTSTRAP_ROUNDS, "estimations_per_round", NUM_ESTIMATIONS_PER_ROUND);

Starting round 425
Starting round 54
Starting round 897
Starting round 949
Starting round 372
Starting round 107
Starting round 689
Starting round 531
Starting round 478
Starting round 1
Starting round 584
Starting round 741
Starting round 793
Starting round 319
Starting round 637
Starting round 845
Starting round 266
Starting round 160
Starting round 213
  round 107 s1 best estimate: fitness 0.0
  round 425 s1 best estimate: fitness 0.0
  round 793 s1 best estimate: fitness 0.0
  round 1 s1 best estimate: fitness 0.0
  round 54 s1 best estimate: fitness 0.0
  round 949 s1 best estimate: fitness 0.0
  round 584 s1 best estimate: fitness 0.0
  round 213 s1 best estimate: fitness 0.0
  round 897 s1 best estimate: fitness 0.0
  round 845 s1 best estimate: fitness 3.1485254740903914e-29
  round 689 s1 best estimate: fitness 0.0
  round 160 s1 best estimate: fitness 3.1143848364195316e-29
  round 372 s1 best estimate: fitness 0.0
  round 478 s1 best estimate: fitness 0.0
  round 531 s1 best